# TAMYETE JUSTICE NTOR - INFORMATION RETRIVAL COURSE WORK

#  Postgraduate Research Search Engine
In this notebook we'll go through all code for building our own search engine specialised on finding potential supervisors for postgraduate research student
Each search engine consists of three main components:


* Crawler
* Indexer
* Query processor



## A. Crawler
We start with building crawler using Beautiful Soap that, extracts lecturers name, Research Area and Link to personal profile from three Universities in the United Kingdom(University of Hull, Kingston University and Swansea University) Processes this information, put in a pandas dataframe and stores it as csv file.

### Import packages

In [1]:
import requests
from requests import get
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
from time import sleep
from random import randint
import re

### University of Hull Crawler

In [2]:
stafflink1=[]
names1 = []
area1= []
links1= []
department = []
page1 = requests.get("https://www.hull.ac.uk/work-with-us/research/our-people")
soup1 = BeautifulSoup(page1.text, 'html.parser')
for main in soup1.find_all('li', class_='item'):
    try:
        slist = main.a['href']
    except Exception as e:
        slist = 'None'
    stafflink1.append(slist)
for ppage in stafflink1:
    ppages = requests.get("https://www.hull.ac.uk" + str(ppage))
    sp1 = BeautifulSoup(ppages.text, 'html.parser')
    try:
        sname= sp1.find('section', class_='profile').h1.text 
    except Exception as e:
        sname = 'None'
    names1.append(sname)    
    try:
        sarea= sp1.find('div', id='research-panel').text
    except Exception as e:
        sarea = 'None'
    area1.append(sarea)
    
    slink = ("https://www.hull.ac.uk" + str(ppage))
    links1.append(slink)
    try:
        dep= sp1.find('div', class_='faculty').li.text
    except Exception as e:
        dep = 'None'
    department.append(dep)
hull = pd.DataFrame({
'name': names1,
'link':links1,
'research_interest': area1
})  
  

### Kingston University Crawler

In [3]:
stafflink2=[]
names2 = []
area2 = []
links2= []
page2 = requests.get("https://www.kingston.ac.uk/staff/search-results/faculty/faculty-of-science-engineering-and-computing-5")
soup2 = BeautifulSoup(page2.text, 'html.parser')
for main2 in soup2.find_all('div', class_='staff-profile-listing-img'):
    try:
        slist2 = main2.a['href']
    except Exception as e:
        slist2 = 'None'
    stafflink2.append(slist2)
for ppage2 in stafflink2:
    ppages2 = requests.get("https://www.kingston.ac.uk" + str(ppage2))
    sp2 = BeautifulSoup(ppages2.text, 'html.parser')
    try:
        sname2=sp2.find('h1', id ='staff-profile-h1').text 
    except Exception as e:
        sname2 = 'None'
    names2.append(sname2)    
    try:
        sarea2=sp2.find('div', class_='single-col-accordian-inner').find_all('div', class_='site-expand-box-content')[1].text
    except Exception as e:
        sarea2 = 'None'
    area2.append(sarea2)
    
    slink2 = ("https://www.kingston.ac.uk" + str(ppage2))
    links2.append(slink2)    
kingston = pd.DataFrame({
'name': names2,
'link':links2,
'research_interest': area2
})

### Swansea University 

In [4]:
stafflinks3=[]
stafflink3=[]
names3 = []
area3 = []
links3= []
stype = ['professors-contents','associate-professors-contents','senior-lecturers-contents','lecturers-contents']
page3 = requests.get("https://www.swansea.ac.uk/staff/engineering/")
soup3 = BeautifulSoup(page3.text, 'html.parser')
for sstype in stype:
    for main3 in soup3.find('div', id =str(sstype)).find_all('p'):
        try:
            slist3 = main3.a['href']
        except Exception as e:
            slist3= None
        stafflinks3.append(slist3)
        stafflink3 = list(filter(None,stafflinks3)) 
def Filter(stafflinks3, substr): 
    return [str for str in stafflinks3 if
             any(sub in str for sub in substr)]
substr =['http']
stafflink3 = Filter(stafflinks3, substr)
for ppage3 in stafflink3:
    ppages3 = requests.get(str(ppage3))
    sp3 = BeautifulSoup(ppages3.text, 'html.parser')
    try:
        sname3=sp3.find('h1', class_='staff-profile-overview-honorific-prefix-and-full-name').text
    except Exception as e:
        sname3 = 'None'
    names3.append(sname3)    
    try:
        sarea3= sp3.find('div', class_='staff-profile-areas-of-expertise').text
    except Exception as e:
        sarea3 = 'None'
    area3.append(sarea3)    
    slink3 = (str(ppage3))
    links3.append(slink3)
swansea = pd.DataFrame({
'name': names3,
'link':links3,
'research_interest': area3
})

Combining all the results and saving the data  

In [5]:
uni =[hull,kingston,swansea]
university = pd.concat(uni)
university['ID'] = [x for x in range(1, len(university.values)+1)]
university.to_csv('university.csv')

In [6]:
university = pd.read_csv("university.csv")
university.drop(university.columns[0], axis=1, inplace=True)
university.head(5)

,name,link,research_interest,ID
0,Miss Amy Tomlinson,https://www.hull.ac.uk/staff-directory/amy-tom...,\n\nResearch interests\nSport rehabilitation \...,1
1,Dr Ireneous Soyiri,https://www.hull.ac.uk/staff-directory/ireneou...,\n\nResearch interests\nMy areas of interests ...,2
2,Christopher Clarke,https://www.hull.ac.uk/staff-directory/christo...,\n\nPostgraduate supervision\nDr Clarke would ...,3
3,Roberto Fernandez Arrieta,https://www.hull.ac.uk/staff-directory/roberto...,\n\nResearch interests\nGeomorphology \nSedime...,4
4,Miss Katherine Bloomfield,https://www.hull.ac.uk/staff-directory/katheri...,\n\nResearch interests\nPublic sector procurem...,5


In [7]:
print(university.dtypes)

# to see where you're missing data and how much data is missing 
print(university.isnull().sum())

name                 object
link                 object
research_interest    object
ID                    int64
dtype: object
name                 0
link                 0
research_interest    0
ID                   0
dtype: int64


In [8]:
university.rename(columns={'name':'names'}, inplace=True)

In [9]:
university

,names,link,research_interest,ID
0,Miss Amy Tomlinson,https://www.hull.ac.uk/staff-directory/amy-tom...,\n\nResearch interests\nSport rehabilitation \...,1
1,Dr Ireneous Soyiri,https://www.hull.ac.uk/staff-directory/ireneou...,\n\nResearch interests\nMy areas of interests ...,2
2,Christopher Clarke,https://www.hull.ac.uk/staff-directory/christo...,\n\nPostgraduate supervision\nDr Clarke would ...,3
3,Roberto Fernandez Arrieta,https://www.hull.ac.uk/staff-directory/roberto...,\n\nResearch interests\nGeomorphology \nSedime...,4
4,Miss Katherine Bloomfield,https://www.hull.ac.uk/staff-directory/katheri...,\n\nResearch interests\nPublic sector procurem...,5
...,...,...,...,...
992,Dr Min Luo,http://www.swansea.ac.uk/staff/engineering/min...,\nAreas Of Expertise\n\nWave hydrodynamics\nTw...,993
993,Dr Christopher Phillips,https://www.swansea.ac.uk/staff/engineering/c....,None,994
994,Dr Elizabeth Sackett,http://www.swansea.ac.uk/staff/engineering/e.s...,\nAreas Of Expertise\n\nFatigue\nCreep\nStrain...,995
995,Dr Andrew Tappenden,https://www.swansea.ac.uk/staff/engineering/a....,\nAreas Of Expertise\n\nMechanical Design\n3D ...,996


## B. Indexer
Second part of a search engine is an indexer. It's basically a smart storage of our data in which we can later easily retrieve data given a search query.
It parses the name and research interest of the data scraped by the crawler to single words. All these words make up the vocabulary of our index. Next step is to put the ID of the data in the posting lists of the words that the data contains. For example data called "This happened today" will be stored in posting lists of terms "this", "happened" and "today".
Before creating the index we preprocess the text of the data in order to get rid of useless information. We the text of accents and turn everything to lowercase. Next we perform lemmatization. This is slightly smarter version of stemming. Essentially, it's a word normalization, e.g. all nouns to singular, all verbs in present tense etc.

Let's do it.

#### Importing

In [10]:
import nltk
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.corpus import wordnet
from nltk import pos_tag
from nltk.stem import WordNetLemmatizer 
import string

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\ntorj\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\ntorj\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ntorj\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ntorj\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [11]:
single_entry = university.loc[0,:].copy()

In [12]:
single_entry

names                                               Miss Amy Tomlinson
link                 https://www.hull.ac.uk/staff-directory/amy-tom...
research_interest    \n\nResearch interests\nSport rehabilitation \...
ID                                                                   1
Name: 0, dtype: object

### Text preprocessing
Turn title to lowercase, remove accents

In [13]:
def process_string(text):
  text = text.lower() #to lowercase
  text = text.translate(str.maketrans('', '', string.punctuation)) #strip punctuation
  return text

In [14]:
process_string(single_entry.research_interest)

'\n\nresearch interests\nsport rehabilitation \n \nrecovery interventions in collision sports \n \nexercise rehabilitation for special populations\n\n'

Now, lemmatize, i.e. word normalization.

This method requires some additional information about the words. We need to find the word category of each word, e.g. verb, noun etc.

In [15]:
def get_wordnet_pos(word):
    """Map POS tag to first character lemmatize() accepts"""
    tag = pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}

    return tag_dict.get(tag, wordnet.NOUN)

Test the function

In [16]:
print("Apple: {}\n Run: {}\n Happy: {}" .format(get_wordnet_pos("apple"), get_wordnet_pos("run"), get_wordnet_pos("happy")))

Apple: n
 Run: v
 Happy: a


We also need to remove stopwords, i.e. words with low informational value.

In [17]:
stop = stopwords.words('english')

Now we'll iterate over all words in text, lemmatize and return the transformed string.

In [18]:
lem = WordNetLemmatizer()

def stop_lemmatize(doc):
    tokens = nltk.word_tokenize(doc)
    tmp = ""
    for w in tokens:
        if w not in stop:
            tmp += lem.lemmatize(w, get_wordnet_pos(w)) + " "
    return tmp

In [19]:
stop_lemmatize(doc = single_entry.research_interest)

'Research interest Sport rehabilitation Recovery intervention collision sport Exercise rehabilitation special population '

In [20]:
def process_string(text):
  text = text.lower() #to lowercase
  text = text.translate(str.maketrans('', '', string.punctuation)) #strip punctuation
  text = stop_lemmatize(text)
  return text

In [21]:
%time process_string(single_entry.research_interest)

Wall time: 8.98 ms


'research interest sport rehabilitation recovery intervention collision sport exercise rehabilitation special population '

Now we apply the process_string function to all names and research interest in our database.

In [22]:
university_processed = university.copy()

In [23]:
def transform_df(df):
  df['names'] = df['names'].apply(process_string)
  df['research_interest'] = df['research_interest'].apply(process_string)

In [24]:
%time transform_df(university_processed)

Wall time: 31.9 s


In [25]:
university_processed.head(5)

,names,link,research_interest,ID
0,miss amy tomlinson,https://www.hull.ac.uk/staff-directory/amy-tom...,research interest sport rehabilitation recover...,1
1,dr ireneous soyiri,https://www.hull.ac.uk/staff-directory/ireneou...,research interest area interest include enviro...,2
2,christopher clarke,https://www.hull.ac.uk/staff-directory/christo...,postgraduate supervision dr clarke would welco...,3
3,roberto fernandez arrieta,https://www.hull.ac.uk/staff-directory/roberto...,research interest geomorphology sediment trans...,4
4,miss katherine bloomfield,https://www.hull.ac.uk/staff-directory/katheri...,research interest public sector procurement pr...,5


Now we can iterate over all entries to create the index. We'll go step by step again before wrapping it all in one nice function.

Merge title and summary into one field and drop all columns except for ID as we don't need those anymore.

In [26]:
university_processed['text'] = university_processed['names'] + " " + university_processed['research_interest']
drop_cols = ['names', 'research_interest', 'link']
university_processed = university_processed.drop(drop_cols, axis=1)

In [27]:
university_processed.dtypes

ID       int64
text    object
dtype: object

Add this part to a transform_df function.

In [28]:
def transform_df(df):
  df = df
  df['names'] = df['names'].apply(process_string)
  df['research_interest'] = df['research_interest'].apply(process_string)
  df['text'] = df['names'] + " " + df['research_interest']
  drop_cols = ['names', 'research_interest', 'link']
  df = df.drop(drop_cols, axis=1)
  return df

### Build index

Now we'll build index with just one entry.

In [29]:
single_entry = university_processed.loc[0,:].copy()
print(single_entry)
index_test = {}

ID                                                      1
text    miss amy tomlinson  research interest sport re...
Name: 0, dtype: object


Split the entry to single words and return list and save entry's ID as object.

In [30]:
words = single_entry.text.split()
ID = single_entry.ID

Each word in index' vocabulary is a dictionary key and has its own posting list with IDs. Let's construct one word vocabulary as example.

In [31]:
word = words[0]
sample = {word: [ID]}
print(sample)

{'miss': [1]}


Now we iterate over all words and if they aren't in the vocabulary yet we add them. Also for each word we append the entry ID to the posting list.

In [32]:
for word in words:
  if word in index_test.keys():
    if ID not in index_test[word]:
      index_test[word].append(ID)
  else:
    index_test[word] = [ID]

In [33]:
print(index_test)

{'miss': [1], 'amy': [1], 'tomlinson': [1], 'research': [1], 'interest': [1], 'sport': [1], 'rehabilitation': [1], 'recovery': [1], 'intervention': [1], 'collision': [1], 'exercise': [1], 'special': [1], 'population': [1]}


Now this process can be repeated for all entries in the database

In [34]:
def index_it(single_entry, index):
  words = single_entry.text.split()
  ID = single_entry.ID
  for word in words:
    if word in index.keys():
      if ID not in index[word]:
        index[word].append(ID)
    else:
      index[word] = [ID]
  return index

In [35]:
ind = index_it(single_entry=single_entry, index= {})
print(ind)

{'miss': [1], 'amy': [1], 'tomlinson': [1], 'research': [1], 'interest': [1], 'sport': [1], 'rehabilitation': [1], 'recovery': [1], 'intervention': [1], 'collision': [1], 'exercise': [1], 'special': [1], 'population': [1]}


Again we can iterate over all entries in the database with scraped articles, process them append to index.

In [36]:
def index_all(df, index):
  for i in range(len(df)):
    single_entry = df.loc[i,:]
    index = index_it(single_entry = single_entry, index = index)
  return index

In [37]:
index = index_all(university_processed, index = {})
len(index)

12199

Finally we wrap everything in one nice function.

In [38]:
def build_index(df, index):
    to_add = transform_df(df)
    index = index_all(df = to_add, index = index)
    return index

In [39]:
idx = build_index(df = university, index = {})

In [40]:
len(idx)

12199

It can be of course opened again with following code.

# Bb Ranked retrieval
  The user would probably prefer the more relevant pages to be displayed before those that are less relevant (hopefully they're at least a bit relevant).
For our search engine to support such option we need to store some information about the scraped documents that could be later used for this purpose.
We'll use averaged word2vec for this purpose. Word2Vec model is single hidden-layer neural network. The hidden layer is actually what is so useful about this model. Given a word the layer's activation gives a unique vector that word. For each document we can iterate over all words, extract their vectors and then by averaging obtain a document vector. 
  Compared to other methods averaged word2vec has multiple advantages. Unlike simpler methods such as bag-of-words, n-grams and tf-idf the size of the vectors is fixed. For example bag-of-words is also using vectors but the size of these vectors equals the number of unique words in the corpus. This means that the computational and storage requirements get larger as the corpus gets larger.
  Averaged word2vec is also able to represent the documents on more abstract level than simpler methods and should therefore provide better method of ranking.
  We're using word2vec rather than doc2vec because we can simply use pretrained word2vec model to compute the document vectors. Using doc2vec would mean training a neural network from scratch which requires computational power, time and rather large dataset. 

Import and download pretrained word2vec model

In [41]:
import gensim
import numpy as np

Load word2vec model

In [42]:
#DOWNLOAD PRETRAINED  NETWORK FROM https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz
word2vec = gensim.models.KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin/GoogleNews-vectors-negative300.bin', binary=True)

Try getting vectors for all words in the text and averaging to get single vector.

In [43]:
print(words)

['miss', 'amy', 'tomlinson', 'research', 'interest', 'sport', 'rehabilitation', 'recovery', 'intervention', 'collision', 'sport', 'exercise', 'rehabilitation', 'special', 'population']


In [44]:
def average_vectors(word2vec_model, doc):
    # remove out-of-vocabulary words
    doc = [word for word in doc if word in word2vec_model.vocab]
    if len(doc) == 0:
      return np.zeros(300)
    else:
      return np.mean(word2vec_model[doc], axis=0)

In [45]:
%time test_vec = average_vectors(word2vec, words)

Wall time: 1.99 ms


Now we can iterate over documents, compute their vectors and construct a document vectors database.

In [46]:
def prepare_ranking(df):
  corpus = df[['ID', 'text']].copy()
  doc_vecs = {}
  for i in range(len(corpus)):
    row = corpus.loc[i,:]
    text = row.text.split()
    doc_vecs[row.ID]=average_vectors(word2vec, text)
  doc_vecs = pd.DataFrame.from_dict(data=doc_vecs, orient="index")
  doc_vecs['ID'] = doc_vecs.index
  return doc_vecs

In [47]:
doc_vecs = prepare_ranking(df=university)
doc_vecs

,0,1,2,3,4,5,6,7,8,9,...,291,292,293,294,295,296,297,298,299,ID
1,-0.033639,0.064815,-0.006880,0.080584,-0.054583,0.065465,0.137582,-0.219918,0.043688,-0.025251,...,-0.012399,0.015765,-0.031241,0.071703,0.015647,-0.041532,-0.047892,-0.039115,-0.045537,1
2,-0.044364,0.009599,0.038593,0.064039,-0.013248,0.032375,0.049146,-0.093369,0.116338,-0.044781,...,0.027362,-0.052934,0.024333,-0.000247,0.056342,-0.016182,-0.019038,0.090509,-0.001769,2
3,-0.133180,0.010523,0.040707,0.026464,-0.021356,0.057363,0.043726,-0.122334,0.088109,0.002358,...,0.097669,-0.025110,0.028538,-0.040394,0.059853,-0.061819,-0.095485,0.004415,0.037643,3
4,0.031822,0.030651,-0.001853,0.008954,-0.149170,0.036586,-0.028792,-0.152965,0.032604,-0.049028,...,-0.002744,-0.063599,-0.001956,-0.001653,0.026438,-0.042178,-0.061358,0.030640,-0.032465,4
5,-0.044116,0.033764,0.019347,0.063199,-0.054460,-0.001245,0.154236,-0.076408,0.088623,-0.015291,...,-0.006299,-0.071753,0.048958,0.042619,0.006645,-0.028208,-0.094398,-0.009499,-0.017594,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
993,-0.032356,0.044762,0.011742,0.083702,-0.119747,0.077774,-0.013001,-0.196220,0.045048,0.062546,...,0.045069,-0.045418,0.052832,0.035725,0.011257,-0.022430,-0.068256,-0.059845,0.045273,993
994,-0.212280,0.095703,-0.007629,0.125153,0.100250,0.004333,-0.016743,-0.174133,0.082458,0.084656,...,0.140930,-0.067017,0.057617,-0.069565,-0.088501,-0.156799,-0.173462,-0.145264,0.147766,994
995,-0.002963,0.106268,-0.039706,0.095398,-0.152854,0.157559,-0.083024,-0.207053,0.112108,0.024325,...,0.087114,-0.074174,0.021534,-0.079060,-0.021340,-0.131270,-0.047857,0.063665,-0.040827,995
996,-0.031346,0.003174,0.009143,0.093140,-0.041583,0.034674,-0.023224,-0.212476,-0.002386,0.029651,...,0.130853,-0.109033,0.126782,-0.016748,-0.017285,-0.057029,-0.151337,-0.085132,-0.012524,996


## C. Query processor
The final part of a search engine is a query processor which actually performs the search task. Given a query by user the processor should return list of relevant documents.
There are multiple types of queries. We'll start with a simple "google-ish" query where assume the user looks for documents relevant to all words in the query. Therefore we transform the query to boolean by connecting all words with AND operator.

First, the processor preprocesses the query the same way as the indexer preprocessed the text. In other words, we normalize the query to match the format of text in the index. Next, the query is parsed to single words. We look into index if these words are part of the vocabulary. If a word is in index we retrieve its posting list. Finally, we look for intersection of all retrieved posting lists. The result is list of document IDs that the user asked for.
However, we need to return something more useful than just a list of IDs. Therefore,we retrieve the information stored about the documents in the university database. Before printing the results we should also rank the documents. This ranking should be based on relevance to query.

-----
To implement:
 - Boolean query
 - phrase matching
 -----


### Normalize query

Let's define an example query.

In [48]:
test = "Christopher Clarke"

Now we use the "process string" function from used by indexer to normalize the query

In [49]:
print("User query: {}." .format(test))
test_norm = process_string(test)
print("Normalized query: {}." .format(test_norm))

User query: Christopher Clarke.
Normalized query: christopher clarke .


Now we split the query into words.

In [50]:
test_split = test_norm.split()

And we wrap this in function

In [51]:
def process_query(query):
  norm = process_string(query)
  return norm.split()

### Retrieve from index

And we iterate over the words, looking if they're in the index vocabulary. If so then we retrieve the associated posting list.

In [52]:
retrieved = []
for word in test_split:
  if word in index.keys():
    retrieved.append(index[word])

Now we look for the intersection of all posting lists.

In [53]:
def lists_intersection(lists):
  intersect = list(set.intersection(*map(set, lists)))
  intersect.sort()
  return intersect

In [54]:
result = lists_intersection(retrieved)
print(result)

[3]


Let's wrap this part in a function before proceeding to formatting the results. The additional if statement is for cases when there's nothing retrieved.

In [55]:
def search_googleish(query, index=idx):
  query_split = process_query(query)
  retrieved = []
  for word in query_split:
    if word in index.keys():
      retrieved.append(index[word])
  if len(retrieved)>0:
    result = lists_intersection(retrieved)
  else:
      result = ['No Information Found']
  return result

In [56]:
result_IDs = search_googleish("virus", index)
print(result_IDs)

['No Information Found']


-----

*TO DO:
If there's no document retrieved, try removing one term and looking for simplified query + tell user that such document doesn't include term X.*

-----

### Retrieve from our database
Now we need to connect the retrieved IDs with some useful information stored in database that we first use to refine the results and then to print nice result to user.

In [57]:

#this is our database
meta = university.drop(['text'], axis=1).copy()
meta.head(5)

,names,link,research_interest,ID
0,miss amy tomlinson,https://www.hull.ac.uk/staff-directory/amy-tom...,research interest sport rehabilitation recover...,1
1,dr ireneous soyiri,https://www.hull.ac.uk/staff-directory/ireneou...,research interest area interest include enviro...,2
2,christopher clarke,https://www.hull.ac.uk/staff-directory/christo...,postgraduate supervision dr clarke would welco...,3
3,roberto fernandez arrieta,https://www.hull.ac.uk/staff-directory/roberto...,research interest geomorphology sediment trans...,4
4,miss katherine bloomfield,https://www.hull.ac.uk/staff-directory/katheri...,research interest public sector procurement pr...,5


Query from database to get only rows of retrieved IDs

In [58]:
def connect_id_df(retrieved_id, df):
    return df[df.ID.isin(retrieved_id)].reset_index(drop=True)

In [59]:
result_meta = connect_id_df(result_IDs, meta)
result_meta.head(5)

,names,link,research_interest,ID


### Ranked retrieval
Now we return back to the word2vec vectors we computed after indexing the documents.
We'l compute the vector for the query as well and then using a cosine similarity compare query to retrieved document relevance.

Compute vector for query

In [60]:
query_vec = average_vectors(word2vec, test_split)

Retrieve vectors of retrieve documents.

In [61]:
result_vecs = connect_id_df(result_IDs, doc_vecs)

Compute cosine similarity between retrieved documents and query

In [62]:
def cos_similarity(a, b):
  dot = np.dot(a, b)
  norma = np.linalg.norm(a)
  normb = np.linalg.norm(b)
  cos = dot / (norma * normb)
  return(cos)

In [63]:
cos_sim = []
for i in range(len(result_vecs)):
  doc_vec = result_vecs.loc[i,:].drop(['ID'])
  cos_sim.append(cos_similarity(doc_vec, query_vec))
result_meta['rank'] = cos_sim

Sort retrieved docs by cosine similarity which is proxi for relevance.

In [64]:
result_meta.sort_values('rank', axis=0)

,names,link,research_interest,ID,rank


Wrap this in function

In [65]:
def rank_results(query, results):
  query_norm = process_query(query)
  query_vec = average_vectors(word2vec, query_norm)
  result_vecs = connect_id_df(results.ID, doc_vecs)
  cos_sim = []
  for i in range(len(result_vecs)):
    doc_vec = result_vecs.loc[i,:].drop(['ID'])
    cos_sim.append(cos_similarity(doc_vec, query_vec))
  results['rank'] = cos_sim
  results = results.sort_values('rank', axis=0)
  return results

In [66]:
final_result = rank_results("Christopher Clarke", result_meta)

### Print results to user

In [67]:
def print_results(result_df):
  for i in range(len(result_df)):
    res = result_df.loc[i, :]
    print(res.names)
    print(res.research_interest)
    if i == len(result_df):
        print(res.link)
    else:
        print("{}\n" .format(res.link))

In [68]:
print_results(final_result)

### Put it all together

In [69]:
def search(query, dat=None):
  result = search_googleish(query)
  result = connect_id_df(result, meta)
  result = rank_results(query, result)
  print_results(result)

In [71]:
query = input("Search for:")
search(query)

Search for:machine learning
dr evdokia tapoglou 
research interest machine learn optimization wave forecasting offshore renewable energy climate change adaptation mitigation groundwater model 
https://www.hull.ac.uk/staff-directory/evdokia-tapoglou

dr igor menezes 
research interest organisational psychology microbehaviours cognitive analytics machine learn psychometrics quantitative method psychological network personality postgraduate supervision organisational climate organisational commitment engagement organisational citizenship behaviour turnover intention reason 
https://www.hull.ac.uk/staff-directory/igor-menezes

dr muhammad awais 
research interest apply ai machine learn deep learn internet thing smart city healthcare industry ict base assistive technology health biomedical signal processing wearable compute data analytics data mining 
https://www.hull.ac.uk/staff-directory/muhammad-awais

dr john atanbori 
research interest computer vision machine learn deep learn applicati